## Configuration

In [22]:
### Configuration
# Dependencies
import os
import pandas as pd
import numpy as np
from summerplaylist import *
from datetime import date

# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

# Old Playlists
playlistsOld = [
    'Summer 2018',
    'Summer 2019',
    'Endless Summer 2020',
    'Endless Summer 2021',
    'Endless Summer 2022',
    'Endless Summer 2023'
]

# Filepaths
dataPath = os.path.join('data', 'artistsKnown.csv')

## Log In

In [2]:
# Authenticate to Spotify
credentials = authenticateToSpotify(CLIENTID, CLIENTSECRET, REDIRECT_URI)

Opening browser for Spotify login...


## Read in Known Artists

In [23]:
# Read in known artists from csv
if os.path.exists(dataPath):
    artistsKnown = pd.read_csv(dataPath)
else:
    artistsKnown = pd.DataFrame(columns=['artistID', 'isLiked'])

## Read in old playlists

In [24]:
# Empty df
artistsOld = pd.DataFrame()
# Read in all old summer playlists
for playlistName in playlistsOld:
    # Get playlistID
    id_pl = getPlaylistID(credentials, playlistName)
    # Get playlistInfo
    tracks_pl = fetchTracksFromPlaylist(credentials, id_pl)
    # Append to df
    artistsOld = pd.concat([artistsOld, tracks_pl], axis=0)
# Deduplicate
artistsOld = artistsOld.drop_duplicates('artistID', ignore_index=True)[['artistID']]

## Merge and deduplicate known artists and any from new playlists

In [25]:
artistsKnown = pd.concat([artistsKnown, artistsOld], axis=0).drop_duplicates('artistID', ignore_index=True)
# Mark Liked and Unliked artists
artistsKnown.loc[artistsKnown['artistID'].isin(artistsOld['artistID']), 'isLiked'] = "Liked"
artistsKnown.loc[~artistsKnown['artistID'].isin(artistsOld['artistID']), 'isLiked'] = "Not Liked"

## Get artist info and add it to historical data

In [ ]:
# Write function to get chunks of artist info
    # Need date, artistID, artistName, artistPopularity, artistFollowers, artistGenre, isLiked

In [26]:
artistsKnown

,artistID,isLiked
0,1URnnhqYAYcrqrcwql10ft,Liked
1,2RTUTCvo6onsAnheUk3aL9,Not Liked
2,41Q0HrwWBtuUkJc7C1Rp6K,Not Liked
3,4NiJW4q9ichVqL1aUsgGAN,Not Liked
4,09hVIj6vWgoCDtT03h8ZCa,Not Liked
...,...,...
1999,3Hb64DQZIhDCgyHKrzBXOL,Liked
2000,229NvsczXKwdH0DkPg4uwT,Liked
2001,0bDqmuVnpecyZQuXZ7i85s,Liked
2002,0Up3H1s9ME6FivFcOz6qte,Liked


## Load

In [ ]:
# Write artistsKnown and artistsHistorical csvs